In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, FastICA
from sklearn.model_selection import LeaveOneOut, KFold, LeaveOneOut, train_test_split
from sklearn.linear_model import LinearRegression, ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, roc_auc_score
import scipy as sp
from scipy.stats import pearsonr
pd.options.mode.chained_assignment = None
from IPython.core.debugger import set_trace
import os
import matplotlib.pyplot as plt
import itertools
import nibabel as nib
import seaborn as sns

%matplotlib inline

In [2]:
# define some functions used later in the main functions
# or are helpers for reading and handling data

def match_dfs_by_ind(df_list, behav, to_compare=[]):
    # if we dont have the same subjects available for all tasks, it matches it
    all_indices = [list(df.index) for df in df_list]
    all_indices.append(behav.index)
    if len(to_compare):
        all_indices.append(to_compare.index)
    in_all = list(set(all_indices[0]).intersection(*all_indices))
    return [df[df.index.isin(in_all)] for df in df_list], behav[behav.index.isin(in_all)]
    
def read_features(to_use, subjlist='test_subjlist', to_scale=True):
    orig_mat = np.genfromtxt(f'data/{to_use}.csv', delimiter=',')
    subjects_path = f'data/{subjlist}.txt'
    with open(subjects_path, 'r') as f:
        subjects = [line.rstrip('\n') for line in f]

    # arrange features in df and keep only subjects that we have g scores for
    features = pd.DataFrame(data=orig_mat, index=subjects)
    
    if to_scale:
        # scale features
        scaler = StandardScaler()
        scaled_features = pd.DataFrame(data = scaler.fit_transform(features), index = features.index)
        
        return scaled_features
    else:
        return features

def decompose(data, num_comps, transformer='pca'):
    # recieves a subjectsXvertices matrix. returns verticesXcomponents matrix
    if transformer == 'pca':
        trans = PCA()
    if transformer == 'ica':
        trans = FastICA(max_iter=500)
    trans.fit(data)
    return trans.components_.T[:,:num_comps]

def demean_by_train(train, test):
    # demeans the data by the training sets mean

    train_avg = train.mean(axis=0)
    train_demeaned = train-train_avg
    test_demeaned = test-train_avg
    return train_demeaned, test_demeaned

def shuffle_copy(to_shuffle):
    # returns shuffled DataFrame to utilize in permutation tests
    
    shuffled = to_shuffle.copy()
    shuffled = shuffled.values
    np.random.shuffle(shuffled)
    return shuffled


def corr_analysis(features, y, ff_num):
    # select featrues cpm-style
    feat_num = features.shape[1]
    corrs = np.zeros(feat_num)
    for feat in range(feat_num):
        corrs[feat] = sp.stats.pearsonr(features[:,feat],y)[0]
    mask = abs(corrs)>=np.sort(abs(corrs))[len(corrs)-ff_num]
    return mask

def save_masked_maps(data, filename):
    template = nib.load('data/Smask.dtseries.nii')
    mask = np.asanyarray(template.dataobj)
    mask[mask==1]=data
    to_save = nib.cifti2.cifti2.Cifti2Image(mask, template.header)
    nib.save(to_save, f'{filename}.dtseries.nii')
    
def get_task_feat_num(mask, num_comps):
    num_tasks = len(mask)/num_comps
    f_per_task = []
    for task in range(int(num_tasks)):
        f_per_task.append((mask[num_comps*task: num_comps*(task+1)].sum()))
    return f_per_task

# two functions - one for single data entry and one for multi

In [3]:
def do_bbs_single(features, score_df, k=10, num_comps=75, score='g_efa', return_stats=True, save_maps=False):
    
    stats = {'r':[], 'mse':[]}
    
    kfold = KFold(n_splits=k, random_state=42, shuffle=True)
    predicted = np.zeros(features.shape[0])
            
    consensus = np.zeros((features.shape[1]))
    
    for fold, (train_index, test_index) in enumerate(kfold.split(features)):
        print(f'fold {fold+1} out of {k}')
        train_features = np.zeros([len(train_index),num_comps])
        test_features = np.zeros([len(test_index),num_comps])
        
        X_train, X_test = features.iloc[train_index,:], features.iloc[test_index,:]
        Y_train, Y_test = score_df[score].iloc[train_index], score_df[score].iloc[test_index]
        # create pca-reduced matrix, in the shpae of verticesXcomponents
        reduced_train = decompose(X_train, num_comps, transformer='pca')

        # extract individual features by calculating expression scores for each subject
        train_features = np.matmul(np.linalg.pinv(reduced_train),X_train.values.T).T            
        test_features = np.matmul(np.linalg.pinv(reduced_train),X_test.values.T).T


        
        # calculate model and get predictions
        model = LinearRegression()
        model.fit(train_features, Y_train)            
        predicted[test_index] = model.predict(test_features)
        
        if return_stats:
            stats['r'].append(sp.stats.pearsonr(predicted[test_index], Y_test)[0])
            stats['mse'].append(mean_squared_error(predicted[test_index], Y_test))
        
        if save_maps:
            # weight components with their related beta values
            weighted_task_comps = reduced_train*model.coef_
            # sum over components to get a single weighted map for this fold
            summed_weighted_task_comps = np.sum(weighted_task_comps,axis=1)
            # add the weighted components to the rest of the weighted components.
            consensus += summed_weighted_task_comps
        
    if return_stats:
        summary = pd.DataFrame(stats).describe().loc[['mean', 'std'],:]
        if save_maps:
            return predicted, stats, summary, consensus
        else:
            return predicted, stats, summary

In [4]:
def do_bbs_multi_data_select(dfs_list, score_df, reg_type, k=10, num_comps=75, score='g_efa', ff_num = 75, l1_ratio=0.01, save_maps=False):
    
    stats = {'r':[], 'mse':[]}

    kfold = KFold(n_splits=k, random_state=42, shuffle=True)
    feature_example = dfs_list[0]
    predicted = np.zeros(feature_example.shape[0])
    
    task_comps_per_fold = []                     
    consensus = np.zeros((feature_example.shape[1]))
    
    for fold, (train_index, test_index) in enumerate(kfold.split(feature_example)):
        print(f'fold {fold+1} out of {k}')
        train_features = np.zeros([len(train_index),num_comps*len(dfs_list)])
        test_features = np.zeros([len(test_index),num_comps*len(dfs_list)])
        # define this folds' Y
        Y_train, Y_test = score_df[score].iloc[train_index], score_df[score].iloc[test_index]

        fold_comps = np.zeros((feature_example.shape[1], num_comps*len(dfs_list)))

        for i in range(len(dfs_list)):
            features = dfs_list[i]
            X_train, X_test = features.iloc[train_index,:], features.iloc[test_index,:]

            # create pca-reduced matrix, in the shpae of verticesXcomponents
            reduced_train = decompose(X_train, num_comps, transformer='pca')

            # extract individual features by calculating expression scores for each subject
            this_train_features = np.matmul(np.linalg.pinv(reduced_train),X_train.values.T).T            
            this_test_features = np.matmul(np.linalg.pinv(reduced_train),X_test.values.T).T
            
            start = i*num_comps
            end = start+num_comps
            train_features[:, start:end] = this_train_features
            test_features[:, start:end] = this_test_features
            
            # save the components used for feature extration 
            if save_maps: 
                fold_comps[:,start:end] = reduced_train
        

        # correlation analysis to select features
        mask = corr_analysis(train_features, Y_train, ff_num)
        task_comps_per_fold.append(get_task_feat_num(mask, num_comps))
        
        # reduce features with mask produced in the correlation analysis
        train_features = train_features[:, mask]
        test_features = test_features[:, mask]
        
        # calculate model and get predictions
        if reg_type == 'glm':
            model = LinearRegression()
        elif reg_type == 'elnet':
            model = ElasticNetCV(l1_ratio=l1_ratio,n_alphas=50, tol=0.001, max_iter=5000)
            
        model.fit(train_features, Y_train)
        betas = model.coef_
                    
        predicted[test_index] = model.predict(test_features)
        
        stats['r'].append(sp.stats.pearsonr(predicted[test_index], Y_test)[0])
        stats['mse'].append(mean_squared_error(predicted[test_index], Y_test))

        if save_maps:
            #reduce fold_comps according to the correlation analysis
            masked_comps = fold_comps[:,mask]
            # weight components with their related beta values
            weighted_task_comps = masked_comps*betas
            # sum over components to get a single weighted map for this fold
            summed_weighted_task_comps = np.sum(weighted_task_comps,axis=1)
            # add the weighted components to the rest of the weighted components.
            consensus += summed_weighted_task_comps
        
    summary = pd.DataFrame(stats).describe().loc[['mean', 'std'],:]
    if save_maps:
        return predicted, stats, summary, consensus
    else:
        return predicted, stats, summary

# examples for how to read in data and run predictions

unfortunately, we cannot upload example data, due to the HCP's terms of conditions.
here we explain how to arange your data and provide the examples for using it with the code in this notebook.


In [6]:
# read in the behavioural data, keep only needed columns
hcp_df = pd.read_csv('hcp_dataframe_with_g.csv')
hcp_df['Subject'] = hcp_df['Subject'].apply(str) 
hcp_df = hcp_df.set_index('Subject')

hcp_df = hcp_df[['PMAT24_A_CR', 'ReadEng_Unadj', 'g_efa', 'NEOFAC_O', 'NEOFAC_C', 'NEOFAC_E', 'NEOFAC_A', 'NEOFAC_N']]
hcp_df = hcp_df.dropna()

In [7]:
# examples for reading in data
subjlist='example_data/test_subjlist'

# here we read in the connectome data - for each subject, a vector of 64620 connectome edges
# created using HCP's Multi Modal Parcellation - https://www.nature.com/articles/nature18933
rs_conn = read_features('example_data/rs_conn',subjlist=subjlist)
# now, read in some task/connTask data
# the data is masked to only include cortex, and aranged such that each subject's map is a row
wm_2bk_orig_z = read_features('example_data/WM_09_s4_z_masked_orig', subjlist=subjlist, to_scale=True)
wm_2bk_pred_z = read_features('example_data/WM_09_s4_z_masked_pred', subjlist=subjlist, to_scale=True)
lang_mathstory_pred_z= read_features('example_data/Lang_03_s4_z_masked_pred', subjlist=subjlist, to_scale=True)


In [8]:
# to run a prediction, we first make sure that the behavioral data includes only the subjects in the image data
dfs, behav = match_dfs_by_ind([wm_2bk_orig_z], hcp_df)
predicted, stats, summary = do_bbs_single(dfs[0], behav)
print(summary)

dfs, behav = match_dfs_by_ind([wm_2bk_pred_z], hcp_df)
predicted, stats, summary = do_bbs_single(dfs[0], behav)
print(summary)

fold 1 out of 10
fold 2 out of 10
fold 3 out of 10
fold 4 out of 10
fold 5 out of 10
fold 6 out of 10
fold 7 out of 10
fold 8 out of 10
fold 9 out of 10
fold 10 out of 10
             r       mse
mean  0.536538  0.523136
std   0.064327  0.082836
fold 1 out of 10
fold 2 out of 10
fold 3 out of 10
fold 4 out of 10
fold 5 out of 10
fold 6 out of 10
fold 7 out of 10
fold 8 out of 10
fold 9 out of 10
fold 10 out of 10
             r       mse
mean  0.425718  0.602067
std   0.081653  0.126863


In [9]:
# now predict using multi data. heres an example on how to do os using 2 images
dfs, behav = match_dfs_by_ind([wm_2bk_pred_z, lang_mathstory_pred_z], hcp_df)
predicted, stats, summary = do_bbs_multi_data_select(dfs, behav, reg_type='elnet')
print(summary)

fold 1 out of 10
fold 2 out of 10
fold 3 out of 10
fold 4 out of 10
fold 5 out of 10
fold 6 out of 10
fold 7 out of 10
fold 8 out of 10
fold 9 out of 10
fold 10 out of 10
             r       mse
mean  0.444150  0.594766
std   0.070218  0.124929


# check significance of comparison between inputs

here we prform significance testing by creating 30 different train-test splits, and on each such split, we perform prediction of a trait, using resting-state connectivity and another different data type with which to compare.
we then test for differnces in prediction accuracy using Wilcoxon signed rank test.

In [20]:
def compare_predictions_single(A,B,hcp_df,score,k=30, num_comps=75):
    dfs, behav = match_dfs_by_ind([A,B], hcp_df)
    
    score_dict = {0:[], 1:[]}
    for i in range(k):
        if i%10==0:
            print(f'iteration {i} out of {k}')
            
        rand_seed = np.random.randint(0,1000)
        for m in [0,1]:
            
            X_train, X_test, y_train, y_test = train_test_split(dfs[m], behav[score], test_size=0.33, random_state=rand_seed)
            reduced_train = decompose(X_train, num_comps, transformer='pca')

            # extract individual features by calculating expression scores for each subject
            train_features = np.matmul(np.linalg.pinv(reduced_train),X_train.values.T).T            
            test_features = np.matmul(np.linalg.pinv(reduced_train),X_test.values.T).T
            model = LinearRegression()
            model.fit(train_features, y_train)
            predicted = model.predict(test_features)
            r = pearsonr(predicted,y_test)[0]
            score_dict[m].append(r)
            
    s,p = sp.stats.wilcoxon(score_dict[0], score_dict[1], alternative='greater')
    return s,p
        

In [21]:
compare_predictions_single(wm_2bk_pred_z, rs_conn, hcp_df, 'g_efa', k=30)

iteration 0 out of 30
iteration 10 out of 30
iteration 20 out of 30


(465.0, 8.666533220995528e-07)

# get data of mean activity in DMN and FPN

this is the code by which we extracted mean activations in DMN and FPN for original and predicted maps, for the analysis presented in figure 3 of the paper

In [ ]:
contrasts = {'2bk>0bk' : 'WM_11_s4', '2bk': 'WM_09_s4', '0bk': 'WM_10_s4',
        'Math-Story' : 'Lang_03_s4',
        'Random' : 'Soc_01_s4', 'TOM': 'Soc_02_s4', 'TOM-Radnom': 'Soc_06_s4',
        'Rel' : 'Rel_02_s4', 'Match': 'Rel_01_s4', 'Rel-Match': 'Rel_04_s4',
        'Reweard': 'Gamb_02_s4', 'Punish': 'Gamb_01_s4', 'Punish-Reward': 'Gamb_03_s4',
        'Faces-Shapes': 'Em_03_s4'}

In [ ]:
yeo_parc = nib.load('/Volumes/HCP/HCP_WB_Tutorial_1.0/yeo_masked.dtseries.nii')
yeo_parc = np.asanyarray(yeo_parc.dataobj)
fpn_num = 6;
dmn_num = 7;
fpn_mask = (yeo_parc==fpn_num).flatten()
dmn_mask = (yeo_parc==dmn_num).flatten()

data_dir = '/Volumes/HCP/Predicted_data_100';

orig_fpn_mean=[]
orig_dmn_mean=[]
pred_fpn_mean=[]
pred_dmn_mean=[]
contrast_description = list(contrasts.keys())
contrast_names = list(contrasts.values())

for con in contrast_names:
    print(con)
    all_orig_path = f'{data_dir}/{con}/all_test_data/all_test_data_orig_z.dtseries.nii'
    all_orig = nib.load(all_orig_path)
    all_orig = np.asanyarray(all_orig.dataobj)
    
    orig_fpn_mean.append(np.mean(np.mean(all_orig[:, fpn_mask])));
    orig_dmn_mean.append(np.mean(np.mean(all_orig[:, dmn_mask])));
    
    all_pred_path = f'{data_dir}/{con}/all_test_data/all_test_data_pred_z_cleaned_cb.dtseries.nii'
    all_pred = nib.load(all_pred_path)
    all_pred = np.asanyarray(all_pred.dataobj)
    
    pred_fpn_mean.append(np.mean(np.mean(all_pred[:, fpn_mask])));
    pred_dmn_mean.append(np.mean(np.mean(all_pred[:, dmn_mask])));
